<a href="https://colab.research.google.com/github/rlagosb/taller_eiv/blob/main/3_Ejercicio_Cubo_multidimensional_(Python).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio: creación de un cubo
En este ejercicio vamos a armar una tabla de datos multidimensional para el análisis de las brechas de consultas de especialidad médica (CNE). Para ello haremos lo siguiente:

1. Generar un modelo estrella con las métricas y dimensiones de análisis

2. Generar tablas fact y dimensiones

3. Exportar a excel

 # Configurar área de trabajo

In [ ]:
# importar librerías
import pandas as pd
import os

# definir carpetas de trabajo
datos_normalizados = 'https://github.com/rlagosb/taller_eiv/raw/refs/heads/main/datos_normalizados/'
datos_cubos = '/content/cubos/'
if not os.path.exists(datos_cubos): os.makedirs(datos_cubos)

# cargar datos
centros = pd.read_parquet(datos_normalizados + 'Centros.parquet')
especialidades = pd.read_parquet(datos_normalizados + 'Especialidades.parquet')
demanda = pd.read_parquet(datos_normalizados + 'Demanda.parquet')
produccion = pd.read_parquet(datos_normalizados + 'Produccion.parquet')

# Crear tabla FACT
Construiremos una tabla con los campos:

- Centro_cod: código del centro de salud
- Especialidad_cod: código de la especialidad médica (prestación)
- Mes: mes del año 2021
- CNE_solicitadas: solicitudes de interconsultas recibidas por establecimiento, especialidad y mes
- CNE_producidas: consultas nuevas de especialidad producidas por establecimientos, especialidad y mes



In [ ]:
# Examinemos los campos que tienen las tablas de demanda y producción
# Con el cruce de estas tablas podemos obtener las métricas fundamentales para el análisis
print(demanda.head(3), end='\n\n')
print(produccion.head(3))

In [ ]:
# Construimos una función para generar la tabla fact

def generar_tabla_fact():

  # Obtener datos de demanda y producción de consultas nuevas
  dda = demanda[['Centro_destino','Especialidad','Mes','Interconsultas']].copy()
  dda = dda.rename(columns={'Centro_destino':'Centro_cod','Especialidad':'Especialidad_cod','Interconsultas':'CNE_solicitadas'})
  prod = produccion[['Centro','Codigo','Mes','Nuevas']].copy()
  prod = prod.rename(columns={'Centro':'Centro_cod','Codigo':'Especialidad_cod','Nuevas':'CNE_producidas'})

  # Cruzar producción y demanda
  fact = dda.merge(prod, how='outer', on = ['Centro_cod','Especialidad_cod','Mes'])

  # Complatemos los campos vacíos con 0 y guardamos las métricas como números enteros
  fact.fillna(0, inplace=True)
  fact[['CNE_solicitadas','CNE_producidas']] = fact[['CNE_solicitadas','CNE_producidas']].astype(int)
  return fact

# Guardamos y revisamos la tabla resultante
fact = generar_tabla_fact()
print(fact.describe)

## 🏁 Discusión
1. ¿Cuáles son las métricas que registra esta tabla?
1. ¿Cuáles son los hechos que registra esta tabla? ¿Cuántos exiten?
2. ¿Cuál es el nivel de detalle con que se describen los hechos (granularidad)?
1. ¿A partir de esta tabla es posible consultar los hechos a nivel diario? ¿Trimestral? ¿Anual?

# Dimensión Periodo
Aunque nuestro análisis sólo considera un año, podríamos querer analizar los datos a nivel trimestral o anual. Construiremos una tabla con los campos

- Mes: llave primaria
- Trimestre
- Año

**⚠** Si trabajáramos con múltiples años necesitaríamos definir otra llave primaria para esta dimensión

In [ ]:
# Creamos un dataframe vacío para agregar los campos
dim_periodo = pd.DataFrame()
dim_periodo['Mes'] = range(1,13)
dim_periodo['Trimestre'] = [1,1,1,2,2,2,3,3,3,4,4,4]
dim_periodo['Año'] = 2021
dim_periodo

# Dimensión Especialidad
Crearemos una tabla con los siguientes campos:

- Especialidad_cod: código de la especialidad (llave primaria)
- Especialidad: nombre de la especialidad
- Alto_riesgo: especialidad con mayor mortalidad para tiempos de espera extensos según Martínez et al ([2019](https://doi.org/10.1186/s12889-019-6526-6))
- REM: si se incluye en reportes estadísticos mensuales


In [ ]:
# Examinemos la tabla especialidades para identificar los campos existentes y los que faltan
especialidades.sample(5)

In [ ]:
# Generemos la dimensión especialidad con una función especial

def crear_dim_especialidades():

  # creamos una copia de especialidades con los campos que necesitamos
  esp = especialidades[['Codigo','Nombre','Codigo_rem']].copy()

  esp.rename(columns={'Codigo':'Especialidad_cod',
                      'Nombre':'Especialidad'},
             inplace = True)

  # Generamos campo REM
  esp['Especialidad_REM'] = 'No'
  esp.loc[esp['Codigo_rem'].notna(), 'Especialidad_REM'] = 'Si'
  del esp['Codigo_rem']

  # Identificamos las especialidades de alto riesgo: CARDIOLOGIA, CIRUGIA CARDIOVASCULAR, GASTROENTEROLOGIA ADULTO, NEFROLOGIA ADULTO, NEUROLOGIA ADULTO, UROLOGIA
  esp['Riesgo_mortalidad'] = 'No'
  alto_riesgo = ['07-003','07-006','07-018','07-037','07-041','07-054']
  esp.loc[esp['Especialidad_cod'].isin(alto_riesgo), 'Riesgo_mortalidad'] = 'Si'

  return esp

dim_especialidad = crear_dim_especialidades()

# Veamos el resultado
print(dim_especialidad.info())
dim_especialidad.sample(5)

## 🏁 Discusión
1. ¿Qué diferencia a esta tabla de la tabla normalizada *Especialidades*?
2. ¿Cuál es el propósito de cada una?

# 🍎 Desafío: Crear Dimensión Establecimiento
Cree un dataframe `dim_centro` con los siguientes campos:

- Centro_cod: código DEIS del centro (llave primaria)
- Centro: nombre del centro de salud
- Complejidad: nivel de complejidad del centro
- Servicio: Servicio de Salud del cual depende

In [ ]:
dim_centro = centros[['Codigo','Nombre','Dependencia','Nivel de Complejidad']].copy()
dim_centro.rename(columns={'Codigo':'Centro_cod',
                           'Nombre':'Centro',
                           'Nivel de Complejidad':'Complejidad',
                           'Dependencia':'Servicio'},
                  inplace=True)
dim_centro

# Cubo
Generamos una tabla multidimensional uniendo la tabla Fact con las dimensiones utilizando las llaves.

In [ ]:
cubo = fact.merge(dim_especialidad,
                    how='left',
                    on = 'Especialidad_cod')

cubo = cubo.merge(dim_centro,
                  how = 'left',
                  on = 'Centro_cod')

cubo = cubo.merge(dim_periodo,
                  how ='left',
                  on = 'Mes')

cubo.sample(5)

In [ ]:
# Exportamos a la carpeta con los cubos
cubo.to_excel(datos_cubos + 'Cubo_brechas_CNE.xlsx', index = False)

## 🏁 Discusión
1. ¿Qué campos tienen redundancias? (¿Cuál forma normal no se cumple?)
3. ¿Cuál es el propósito de introducir esta redundancia?
4. ¿Existe el riesgo que se produzan inconsistencias en los nombres de los centros o especialidades?
1. ¿Qué cambios tendría que hacer a esta tabla si tuviera:
  - Datos del 2021 al 2024?
  - Datos de más establecimientos?